In [1]:
from nilearn import datasets
import pickle
import json
import numpy as np
import networkx as nx
# from scipy import stats
from statsmodels.stats import nonparametric
from multiprocessing import Pool
import os
from esfMRI import stats_tests, plot_graph_measures
from graph import multi_process

In [2]:
filename = "60_3"
os.makedirs(f"graph_theory/{filename}", exist_ok=True)
atlas_labels = datasets.fetch_atlas_aal()['labels']
threshold = 50

In [5]:
with open(f"dFC/{filename}dFCs.pkl", "rb") as f:
    dFCs = pickle.load(f)

In [6]:
save_path = f"graph_theory/pkl/{filename}"
os.makedirs(save_path, exist_ok=True)
with Pool(5) as pool:
    tasks = pool.starmap(multi_process, [[dFCs[sub], sub, threshold, save_path] for sub in dFCs])
graphs = {sub:graph for graph, sub in tasks}
print("全部计算完成，正在保存...")
with open(f"graph_theory/pkl/{filename}_graphs.pkl", "wb") as f:
    pickle.dump(graphs, f)
print("保存完成")

全部计算完成，正在保存...
保存完成


In [3]:
with open(f"graph_theory/pkl/{filename}_graphs.pkl", "rb") as f:
    graphs = pickle.load(f)

In [4]:
keys = ['k_core', 'clustering', 'transitivity', 'degree_assortativity_coefficient', 'global_efficiency', 'local_efficiency', 'ratio']

In [5]:
# 对每个参数、每个被试应用统计学分析
no_changed = {}
change = {}
for key in keys:
    no_changed[key] = []
    change[key] = {}
    for sub in graphs:
        measures_preop = []
        measures_postop = []
        for run in graphs[sub]["ses-preop"]:
            for graph in graphs[sub]["ses-preop"][run]:
                if key == "clustering":
                    measures_preop.append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_preop.append(graph[key])
        for run in graphs[sub]["ses-postop"]:
            for graph in graphs[sub]["ses-postop"][run]:
                if key == "clustering":
                    measures_postop.append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_postop.append(graph[key])
        p_values = stats_tests(measures_preop, measures_postop, False)
        if np.any(p_values[4:5] > 0.05):
            no_changed[key].append(sub)
        else:
            change[key][sub] = {}
            change[key][sub]["rank_cmp"] = nonparametric.rank_compare_2indep(measures_preop, measures_postop).tuple
            change[key][sub]["change"] = "up" if change[key][sub]["rank_cmp"][0] < 0 else "down"
with open(f"graph_theory/{filename}/no_changed_dFC.json", "w") as f:
    json.dump(no_changed, f, indent=4)
with open(f"graph_theory/{filename}/changed_dFC.json", "w") as f:
    json.dump(change, f, indent=4)

In [6]:
for key in keys:
    cnt = 0
    for sub in change[key]:
        cnt += change[key][sub]["change"] == "up"
    print("up:", cnt, "down: ", len(graphs)-cnt)

up: 5 down:  15
up: 5 down:  15
up: 8 down:  12
up: 11 down:  9
up: 3 down:  17
up: 4 down:  16
up: 17 down:  3


In [7]:
# 对每个参数整体应用统计学分析
for key in keys:
    measures_preop = []
    measures_postop = []
    for sub in graphs:
        for run in graphs[sub]["ses-preop"]:
            for graph in graphs[sub]["ses-preop"][run]:
                if key == "clustering":
                    measures_preop.append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_preop.append(graph[key])
        for run in graphs[sub]["ses-postop"]:
            for graph in graphs[sub]["ses-postop"][run]:
                if key == "clustering":
                    measures_postop.append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_postop.append(graph[key])
    p_values = stats_tests(measures_preop, measures_postop, False)
    if np.any(p_values[4:5] > 0.05):
        print(key, "无明显变化", p_values)
    else:
        print(key)
        print(nonparametric.rank_compare_2indep(measures_preop, measures_postop).tuple)
    print()

k_core
(20.067036742817777, 1.3866987183190551e-86)



c:\Users\fjj\anaconda3\lib\site-packages\scipy\stats\morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


clustering
(21.48352990957754, 1.5295027909914953e-98)

transitivity
(12.611725884229802, 5.6117992073869923e-36)

degree_assortativity_coefficient
(-7.1794330080584885, 7.865168747756571e-13)

global_efficiency
(28.277420131433914, 7.142553322437408e-165)

local_efficiency
(25.086763226475764, 7.60117606626352e-132)

ratio
(-28.887537828127023, 4.540583295117324e-172)



In [8]:
for key in keys:
    save_path = f"graph_theory/{filename}/{key}"
    os.makedirs(save_path, exist_ok=True)
    for sub in graphs:
        measures_preop = {}
        measures_postop = {}
        for run in graphs[sub]["ses-preop"]:
            measures_preop[run] = []
            for graph in graphs[sub]["ses-preop"][run]:
                if key == "clustering":
                    measures_preop[run].append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_preop[run].append(graph[key])
        for run in graphs[sub]["ses-postop"]:
            measures_postop[run] = []
            for graph in graphs[sub]["ses-postop"][run]:
                if key == "clustering":
                    measures_postop[run].append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_postop[run].append(graph[key])
        plot_graph_measures(measures_preop, measures_postop, key, f"{save_path}/{sub}.png")